
# Algoritmo de triangulação

A triangulação do polígono é o primeiro ingrediente da nossa solução para o problema da galeria de arte. O objetivo desse notebook é explicar como fazer isso através do algoritmo de *ear decomposition* (decomposição por orelhas).  
Primeiro, lemos o polígono de um arquivo de entrada

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random as rn
rn.seed()
# Selecione o polígono entre in0, in1, in2, ... in6
polygon = "in5"

# Leitura da entrada

with open("pols/" + polygon) as f:
    L = list(f.readline().split())

N = int(L[0])
L = L[1:]

points = []

for i in range(0, N):
    s, t = L[2*i], L[2*i+1]
    a, b = map(int, s.split('/'))
    c, d = map(int, t.split('/'))
    # Perturbação para evitar pontos colineares
    eps1, eps2 = rn.random()/1e4, rn.random()/1e4
    points.append((a/b + eps1, c/d+eps2))


# Plotar o polígono

def plt_pol():
    plt.figure(figsize=(5, 5))
    plt.axis('equal')
    xs = [x for (x, y) in points]
    ys = [y for (x, y) in points]
    plt.fill(xs, ys)
    plt.show()

plt_pol()

Precisamos de algumas primitivas geométricas

In [ ]:
# Primitiva de sentido horário/anti-horário


def ccw(i, j, k, eps=1e-9):
    (ix, iy) = points[i]
    (jx, jy) = points[j]
    (kx, ky) = points[k]
    
    (vx, vy) = (jx - ix, jy - iy)
    (wx, wy) = (kx - jx, ky - jy)
    
    return vx*wy - vy*wx > eps


# Primitiva de estar dentro de um triângulo (l dentro de i, j, k)

def inside(i, j, k, l):
    return ccw(i, j, l) and ccw(j, k, l) and ccw(k, i, l)

Segue o algoritmo que decide se um ponto é uma orelha

In [ ]:
# Sou ponta de orelha? Complexidade O(N)

def eartest(j):
    i, k = prev[j], prox[j]
    
    if not ccw(i, j, k):
        return False
    
    for l in range(0, N):
        if inside(i, j, k, l):
            return False

    return True

Segue o algoritmo de triangulação

In [ ]:
# Triangulação. Complexidade: O(N^2)

def triangulate():  
    
    # Adjacência no polígono
    
    global prev, prox
    prev, prox = [], []
    for i in range(0, N):
        prev.append((i-1)%N)
        prox.append((i+1)%N)
    
    # Inicialmente, quem é orelha?
    
    ear = []
    
    for j in range(0, N):
        ear.append(eartest(j))
    
    triangulation = []
    remaining = N
    
    # Loop principal
    
    while remaining >= 3:
    
        # Encontrar uma orelha
        i = -1
        for j in range(0, N):
            if ear[j]:
                i = j
                break
        
        # Adicionar o triangulo encontrado
        triangulation.append((prev[i], i, prox[i]))
        ear[i] = False
        
        # Remover i do polígono
        prox[prev[i]] = prox[i]
        prev[prox[i]] = prev[i]
        remaining -= 1
        
        # Atualizar "status de orelha" dos vizinhos
        ear[prev[i]] = eartest(prev[i])
        ear[prox[i]] = eartest(prox[i])
        
    return triangulation

Agora, vamos plotar o resultado

In [ ]:
def plt_tring():
    plt.figure(figsize=(10, 10))
    plt.axis('equal')
    for t in triangulate():
        l = [points[i] for i in t]
        txs = [x for (x, y) in l]
        tys = [y for (x, y) in l]
        plt.fill(txs, tys, edgecolor='black', linewidth=0.1)
    plt.show()

def tring_anim(i):
    if(i==0):
        return arts
    t = tt[i-1]
    for p in range(1):
        l = [points[i] for i in t]
        txs = [x for (x, y) in l]
        tys = [y for (x, y) in l]
        pol = plt.fill(txs, tys, edgecolor='black', linewidth=0.1)[0]
        arts.append(pol)
    return arts

def anim():
    global tt, arts
    tt = triangulate()
    fig = plt.figure(figsize=(8, 8))
    xs, ys = [x for (x, y) in points], [y for (x, y) in points]
    ax = fig.add_subplot(autoscale_on=False, xlim=(min(xs), max(xs)), ylim=(min(ys), max(ys)))
    ax.set_aspect('equal')
    arts = [plt.fill(xs, ys, color='black')[0]]
    global ani
    ani = animation.FuncAnimation(fig, tring_anim, N-1, interval=1e4/N, blit=True, repeat=False)
    plt.show()

anim()